# 初始化模型参数

我们仍然用MLP这个例子来详细解释如何初始化模型参数。

In [1]:
from mxnet.gluon import nn
from mxnet import nd

def get_net():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(4, activation="relu"))
        net.add(nn.Dense(2))
    return net

x = nd.random.uniform(shape=(3,5))

我们知道如果不`initialize()`直接跑forward，那么系统会抱怨说参数没有初始化。

In [2]:
import sys
try:
    net = get_net()
    net(x)
except RuntimeError as err:
    sys.stderr.write(str(err))

Parameter sequential0_dense0_weight has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

正确的打开方式是这样

In [3]:
net.initialize()
net(x)


[[ 0.00212593  0.00365805]
 [ 0.00161272  0.00441845]
 [ 0.00204872  0.00352518]]
<NDArray 3x2 @cpu(0)>

## 访问模型参数

之前我们提到过可以通过`weight`和`bias`访问`Dense`的参数，他们是`Parameter`这个类：

In [4]:
net

Sequential(
  (0): Dense(5 -> 4, Activation(relu))
  (1): Dense(4 -> 2, linear)
)

In [5]:
w = net[0].weight
b = net[0].bias
print('name: ', net[0].name, '\nweight: ', w, '\nbias: ', b)

name:  sequential0_dense0 
weight:  Parameter sequential0_dense0_weight (shape=(4, 5), dtype=<class 'numpy.float32'>) 
bias:  Parameter sequential0_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)


然后我们可以通过`data`来访问参数，`grad`来访问对应的梯度

In [6]:
print('weight:', w.data())
print('weight gradient', w.grad())
print('bias:', b.data())
print('bias gradient', b.grad())

weight: 
[[-0.06206018  0.06491279 -0.03182812 -0.01631819 -0.00312688]
 [ 0.0408415   0.04370362  0.00404529 -0.0028032   0.00952624]
 [-0.01501013  0.05958354  0.04705103 -0.06005495 -0.02276454]
 [-0.0578019   0.02074406 -0.06716943 -0.01844618  0.04656678]]
<NDArray 4x5 @cpu(0)>
weight gradient 
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
<NDArray 4x5 @cpu(0)>
bias: 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>
bias gradient 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


我们也可以通过`collect_params`来访问`Block`里面所有的参数（这个会包括所有的子`Block`）。它会返回一个名字到对应 `Parameter` 的 `dict`。既可以用正常`[]`来访问参数，也可以用`get()`，它不需要填写名字的前缀。

In [6]:
params = net.collect_params()
print(params)
print(params['sequential0_dense0_bias'].data())
print(params.get('dense0_weight').data())

sequential0_ (
  Parameter sequential0_dense0_weight (shape=(4, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense1_weight (shape=(2, 4), dtype=<class 'numpy.float32'>)
  Parameter sequential0_dense1_bias (shape=(2,), dtype=<class 'numpy.float32'>)
)

[ 0.  0.  0.  0.]
<NDArray 4 @cpu(0)>

[[-0.06206018  0.06491279 -0.03182812 -0.01631819 -0.00312688]
 [ 0.0408415   0.04370362  0.00404529 -0.0028032   0.00952624]
 [-0.01501013  0.05958354  0.04705103 -0.06005495 -0.02276454]
 [-0.0578019   0.02074406 -0.06716943 -0.01844618  0.04656678]]
<NDArray 4x5 @cpu(0)>


## 使用不同的初始函数来初始化

我们一直在使用默认的`initialize`来初始化权重（除了指定GPU `ctx`外）。它会把所有权重初始化成在`[-0.07, 0.07]`之间均匀分布的随机数。我们可以使用别的初始化方法。例如使用均值为 $0$，方差为 $0.02$ 的正态分布

In [8]:
from mxnet import init
params.initialize(init=init.Normal(sigma=0.02), force_reinit=True)
print(net[0].weight.data(), net[0].bias.data())


[[ 0.02804598  0.00220872  0.00701151  0.02721515  0.00500832]
 [ 0.00112992  0.03227538 -0.01813176 -0.00385197 -0.01286032]
 [ 0.03360647 -0.02855298 -0.03083278 -0.02110904 -0.02623654]
 [-0.00293494  0.01282986 -0.01476416  0.04062728  0.01186533]]
<NDArray 4x5 @cpu(0)> 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


看得更加清楚点：

In [9]:
params.initialize(init=init.One(), force_reinit=True)
print(net[0].weight.data(), net[0].bias.data())


[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
<NDArray 4x5 @cpu(0)> 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


更多的方法参见 [init的API](https://mxnet.incubator.apache.org/api/python/optimization.html#the-mxnet-initializer-package). 

## 延后的初始化

我们之前提到过 Gluon 的一个便利的地方是模型定义的时候不需要指定输入的大小，在之后做 `forward` 的时候会自动推测参数的大小。我们具体来看这是怎么工作的。

新创建一个网络，然后打印参数。你会发现两个全连接层的权重的形状里都有 $0$。 这是因为在不知道输入数据的情况下，我们无法判断它们的形状。

In [10]:
net = get_net()
net.collect_params()

sequential1_ (
  Parameter sequential1_dense0_weight (shape=(4, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_weight (shape=(2, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_bias (shape=(2,), dtype=<class 'numpy.float32'>)
)

然后我们初始化

In [11]:
net.initialize()
net.collect_params()

sequential1_ (
  Parameter sequential1_dense0_weight (shape=(4, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_weight (shape=(2, 0), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_bias (shape=(2,), dtype=<class 'numpy.float32'>)
)

你会看到我们形状并没有发生变化，这是因为我们仍然不能确定权重形状。真正的初始化发生在我们看到数据时。

In [12]:
net(x)
net.collect_params()

sequential1_ (
  Parameter sequential1_dense0_weight (shape=(4, 5), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense0_bias (shape=(4,), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_weight (shape=(2, 4), dtype=<class 'numpy.float32'>)
  Parameter sequential1_dense1_bias (shape=(2,), dtype=<class 'numpy.float32'>)
)

这时候我们看到 shape 里面的 $0$ 被填上正确的值了。

## 共享模型参数

有时候我们想在层之间共享同一份参数，我们可以通过 `Block` 的 `params` 输出参数来手动指定参数，而不是让系统自动生成。

In [13]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(4, activation="relu"))
    net.add(nn.Dense(4, activation="relu"))
    net.add(nn.Dense(4, activation="relu", params=net[-1].params))
    net.add(nn.Dense(2))

初始化然后打印

In [14]:
net.initialize()
net(x)
print(net[1].weight.data())
print(net[2].weight.data())


[[-0.00816047 -0.03040703  0.06714214 -0.05317248]
 [-0.01967777 -0.02854037 -0.00267491 -0.05337812]
 [ 0.02641256 -0.02548236  0.05326662 -0.01200318]
 [ 0.05855297 -0.06101935 -0.0396449   0.0269461 ]]
<NDArray 4x4 @cpu(0)>

[[-0.00816047 -0.03040703  0.06714214 -0.05317248]
 [-0.01967777 -0.02854037 -0.00267491 -0.05337812]
 [ 0.02641256 -0.02548236  0.05326662 -0.01200318]
 [ 0.05855297 -0.06101935 -0.0396449   0.0269461 ]]
<NDArray 4x4 @cpu(0)>


## 自定义初始化方法

下面我们自定义一个初始化方法。它通过重载 `_init_weight` 来实现不同的初始化方法。（注意到 Gluon 里面 `bias` 都是默认初始化成 $0$）

In [15]:
class MyInit(init.Initializer):
    def __init__(self):
        super().__init__()
        self._verbose = True
    def _init_weight(self, _, arr):
        # 初始化权重，使用 out = arr 后我们不需指定形状
        print('init weight', arr.shape)
        nd.random.uniform(low=5, high=10, out=arr)

net = get_net()
net.initialize(MyInit())
net(x)
net[0].weight.data()

init weight (4, 5)
init weight (2, 4)



[[9.605788  7.8797326 5.4155626 9.646481  6.3885927]
 [6.592845  5.0467834 8.337051  9.21171   5.6589894]
 [8.23587   8.581636  9.20693   6.4470305 6.323651 ]
 [5.915957  6.989104  7.9325647 7.764107  5.100538 ]]
<NDArray 4x5 @cpu(0)>

当然我们也可以通过 `Parameter.set_data` 来直接改写权重。注意到由于有延后初始化，所以我们通常可以通过调用一次 `net(x)` 来确定权重的形状先。

In [16]:
net = get_net()
net.initialize()
net(x)

print('default weight:', net[1].weight.data())

w = net[1].weight
w.set_data(nd.ones(w.shape))

print('init to all 1s:', net[1].weight.data())

default weight: 
[[ 0.06699993  0.0279271  -0.05373173 -0.02835883]
 [ 0.03738332  0.0439317  -0.01234518 -0.0144892 ]]
<NDArray 2x4 @cpu(0)>
init to all 1s: 
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
<NDArray 2x4 @cpu(0)>


## 总结

我们可以很灵活地访问和修改模型参数。

## 练习

1. 研究下`net.collect_params()`返回的是什么？`net.params`呢？
1. 如何对每个层使用不同的初始化函数
1. 如果两个层共用一个参数，那么求梯度的时候会发生什么？

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/987)